# Assignment 2

In [10]:
#Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.stats import norm
%matplotlib inline

### Load Dataset

In [11]:
#Import Data
stock_data = pd.read_excel('PS1-2_Studynet.xlsx')

#Clean Header
stock_data.columns = stock_data.columns.str.replace("Equity", "")
stock_data.columns = stock_data.columns.str.replace("SE", "")
stock_data.columns = stock_data.columns.str.replace("SW", "")
stock_data.columns = stock_data.columns.str.replace(" ", "")
stock_data = stock_data.set_index('Date')
stock_data = stock_data[['LISN','BAER']].loc['2010-01-01':'2017-12-29']

stock_data['LISN'] = stock_data.LISN / stock_data.LISN.shift(1) - 1
stock_data['BAER'] = stock_data.BAER / stock_data.BAER.shift(1) - 1
stock_data.head()

,LISN,BAER
Date,,
2010-01-01,NaN,NaN
2010-01-08,0.018894,-0.005909
2010-01-15,-0.022600,0.042174
2010-01-22,0.035573,-0.064639
2010-01-29,0.003817,0.000290


### Excercise 1.1

In [19]:
#m_lisn = np.mean(stock_data['LISN'])
#m_baer = np.mean(stock_data['BAER'])

def mad(data):
    return np.mean(np.absolute(data - np.mean(data)))

def semidev(data):
    return np.mean(np.power(np.minimum(stock_data['BAER'],0),2))

def exp_shortfall(alpha,data):
    #condiational variance, expected shortfall
    
    mu = np.mean(data)
    sigma = np.std(data)
    
    return alpha**-1 * norm.pdf(norm.ppf(alpha))*sigma - mu

In [20]:
output = {'LISN':   [round(np.std(stock_data['LISN']),4), mad(stock_data['LISN']), semidev(stock_data['LISN']), exp_shortfall(0.05,stock_data['LISN']), exp_shortfall(0.01,stock_data['LISN'])], 
          'BAER':   [round(np.std(stock_data['BAER']),4), mad(stock_data['BAER']), semidev(stock_data['BAER']), exp_shortfall(0.05,stock_data['BAER']), exp_shortfall(0.01,stock_data['BAER'])]}

ouput_table = pd.DataFrame(data = output, index = ['Std', 'Mad','Semi Dev', 'Exp. Shortfall 95%','Exp. Shortfall 99%'])
ouput_table

,LISN,BAER
Std,0.022400,0.038100
Mad,0.017053,0.026997
Semi Dev,0.000711,0.000711
Exp. Shortfall 95%,0.043561,0.076514
Exp. Shortfall 99%,0.057074,0.099441


### Excersise 1.2

In [49]:
def emp_expshort(data, alpha):
    
    def delta(ret, vara):
        if ret < (vara*-1):
            return 1
        else:
            return 0
    
    def varalpha(data, alpha):
        return - (np.mean(data) - norm.ppf(0.05)*np.std(data))
    
    vara = varalpha(data, alpha)
    
    sum_delta = []
    
    for i in range(0,len(data)):
        sum_delta.append(delta(data[i], vara))

    ES = (-1/np.sum(sum_delta))*np.sum(sum_delta*data)
    
    return ES 

emp_expshort(stock_data['LISN'], 0.05)*100


-0.04961411644774286